In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FHZ8NL37Q to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-144989
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-144989


In [2]:

from azureml.core.compute import ComputeTarget, AmlCompute
compute_cluster_name = "lr-cluster"
provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, compute_cluster_name, provisioning_config)
compute_cluster.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [31]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
        '--C': uniform(0.1, 0.9), '--max_iter': choice(100, 150, 250, 300)})

#ps = BayesianParameterSampling({
#    '--C': uniform(0.1, 1.0), '--max_iter': choice(100, 150, 250)})

# Specify a Policy
#policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

policy = MedianStoppingPolicy(evaluation_interval=2, delay_evaluation=1)

if "training" not in os.listdir():
    os.mkdir("./training")
    shutil.copy("train.py", "./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = ".", entry_script = "train.py", compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                     estimator=est,
                                     max_concurrent_runs=5,
                                     max_total_runs = 20,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE)

In [32]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
hyperdrive_run = exp.submit(hyperdrive_config, showOutput=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [33]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n --C:',parameter_values[1])
print('\n --max_iter',parameter_values[3])
### YOUR CODE HERE ###

Best Run Id:  HD_84771e15-72fa-42a0-befd-d884be71545b_0

 Accuracy: 0.9107044325915027

 --C: 0.6604121292247661

 --max_iter 100


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_to_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path_to_data)

In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=11)
training_data = x_train.join(y_train)

In [19]:
# convert training_dtaa into a TabularDataset because AutoML does not accept pandas dataframe (ConfigException: Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset])
if "data" not in os.listdir():
    os.mkdir("./data")
training_data.to_csv("data/train_data.csv", index=False)
default_data_store = ws.get_default_datastore()
default_data_store.upload(src_dir="./data", overwrite=True, show_progress=True)
training_data_tabular = TabularDatasetFactory.from_delimited_files(path=default_data_store.path('train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [21]:
print(type(training_data_tabular))

<class 'azureml.data.tabular_dataset.TabularDataset'>


In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data_tabular,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_cluster)

In [23]:
# Submit your automl run
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "automl_classification")
run = experiment.submit(automl_config, show_output = True)
RunDetails(run).show()

Submitting remote run.
No run_configuration provided, running on lr-cluster with default configuration
Running on remote compute: lr-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl_classification,AutoML_c2e1bdd9-0322-4401-a210-65d02ba35d36,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [28]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automl_classification,
Id: AutoML_c2e1bdd9-0322-4401-a210-65d02ba35d36_26,
Type: azureml.scriptrun,
Status: Completed)
None


In [30]:
best_run_metrics = best_run.get_metrics()
print("Accuracy: ", best_run_metrics['accuracy'])

Accuracy:  0.9172856656326424
